- input  : 'data' , params{'shape':(3,10,10)}

- layer_1: 'conv' , - params{'input_channel':3,'output_channel':8,'kernel_size':3,'pad':1,'stride':1,'group':1}

- layer_2: 'batchnorm' , params{}

- layer_3: 'relu'

- layer_4: 'conv_dw', 
params{'input_channel':8,'output_channel':8,'kernel_size':3,'pad':1,'stride':1,'group':8}

- layer_5: 'conv_dw_batchnorm', params{}

- layer_6: 'conv_dw_relu'

- layer_7: 'conv', 
params{'input_channel':8,'output_channel':16,'kernel_size':1,'pad':1,'stride':1,'group':1}

In [1]:
import mxnet as mx

import numpy as np

from mxnet import ndarray as nd

import caffe

from collections import namedtuple

/Library/Python/2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### set caffe made

In [2]:
caffe.set_mode_cpu()
ctx = mx.cpu()

#### load model using mxnet

In [642]:
#mxnet load pretrained model
syms, args, auxs = mx.model.load_checkpoint('test_net', epoch=1)

In [655]:
internals = syms.get_internals()
conv1 = internals[2]
conv2_dw_batchnorm = internals[15]
conv2 = internals[17]
full_connected = internals[21]
group = mx.symbol.Group([full_connected, conv2_dw_batchnorm, conv2, syms])

In [656]:
mod = mx.mod.Module(symbol=group, context=ctx)
mod.bind(for_training=False, data_shapes=[('data',(1,3,20,20))])
mod.set_params(arg_params=args, aux_params=auxs)

#### load prototxt and init params using mx model

In [657]:
net = caffe.Net('sample_net_deploy.prototxt', caffe.TEST)

net.params['conv1'][0].data[...] = args['conv1_weight'].asnumpy()

net.params['conv1_batchnorm'][0].data[...] = np.array([0,0])#auxs['conv1_batchnorm_moving_mean'].asnumpy()

net.params['conv1_batchnorm'][1].data[...] = np.array([1,1])#auxs['conv1_batchnorm_moving_var'].asnumpy()

net.params['conv1_batchnorm'][2].data[...] = np.array([1])#args['conv1_batchnorm_gamma'].asnumpy()[0]

net.params['conv2_dw'][0].data[...] = args['conv2_dw_weight'].asnumpy()

net.params['conv2_dw_batchnorm'][0].data[...] = np.array([0,0])#auxs['conv2_dw_batchnorm_moving_mean'].asnumpy()

net.params['conv2_dw_batchnorm'][1].data[...] = np.array([1,1])#auxs['conv2_dw_batchnorm_moving_var'].asnumpy()

net.params['conv2_dw_batchnorm'][2].data[...] = np.array([1])#args['conv2_dw_batchnorm_gamma'].asnumpy()[0]

net.params['conv2'][0].data[...] = args['conv2_weight'].asnumpy()

net.params['fc'][0].data[...] = args['fullyconnected0_weight'].asnumpy()

net.params['fc'][1].data[...] = args['fullyconnected0_bias'].asnumpy()

#### numpy save array to file

data = np.random.rand(1,3,20,20)

np.save('test_data', data)

In [659]:
data = np.load('test_data.npy')

#### caffe net forward

In [660]:
net.blobs['data'].data[...] = data

out = net.forward()

print out

{'InnerProduct1': array([[ 8.2888597e-08,  6.5315042e-09, -4.4180632e-09, -6.2771512e-09]],
      dtype=float32)}


#### mxnet mod predict

In [661]:
out = mod.predict(mx.io.NDArrayIter(data, batch_size=1))

#### mxnet mod forward

In [662]:
Batch = namedtuple('Batch',['data'])

mod.forward(Batch([nd.array(data)]))

out = mod.get_outputs()

#nd.softmax(nd.array(out['InnerProduct1']))